# RoboVision Planner

**Overview**  
RoboVision Planner is a compact multi-agent system that converts an input image and a natural-language instruction into a step-by-step robot action plan. This notebook demonstrates the end-to-end pipeline using Google ADK concepts: VisionTool (tool), VisionAgent, PlannerAgent (LLM), Orchestrator, InMemorySessionService, and a small evaluation harness.



In [ ]:
# --- INSTALLATION ---
!pip install google-adk --upgrade > /dev/null 2>&1

import os
import json
from typing import Any, Dict, List
from datetime import datetime
import uuid
import base64
import asyncio

# ADK Core Imports
from google.genai import types
from google.adk.agents import LlmAgent, Agent, SequentialAgent
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.adk.models.google_llm import Gemini

print("✅ ADK Installed and Imported")


**Authentiucation**

In [ ]:
# --- API KEY SETUP ---
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("🔐 API Key Loaded Successfully")
except:
    print("⚠️ Could not load API Key. Ensure it's set correctly.")


**Global Config**

In [ ]:
# --- GLOBAL CONFIG ---
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

session_service = InMemorySessionService()

GEMINI_MODEL = Gemini(
    model_name="gemini-2.5-flash",
    retry_config=retry_config
)

def pretty_print_json(data: Any):
    print(json.dumps(data, indent=2, ensure_ascii=False))

print("⚙️ Configuration Ready")


## Vision Agent & Planner Agent

We implement:
- **VisionAgent** — calls the VisionToolWrapper to extract a structured scene summary.
- **PlannerAgent** — uses Gemini (or a heuristic fallback) to convert the scene summary and user instruction into a step-by-step plan.




In [ ]:
# --- Agents (VisionAgent, PlannerAgent, Orchestrator) ---

# Create VisionTool instance (uses GEMINI_MODEL if available)
vision_tool = None
try:
    vision_tool = VisionToolWrapper(gemini_model=GEMINI_MODEL, use_real=USE_REAL_VISION)
except Exception as e:
    logger.warning("Could not instantiate VisionToolWrapper cleanly: %s", str(e))
    vision_tool = VisionToolWrapper(gemini_model=None, use_real=False)

# Helper: compact scene summary
def compact_scene(vision_result: Dict[str, Any]) -> Dict[str, Any]:
    objs = vision_result.get('objects', [])
    return {
        'objects': [{'name': o.get('label'), 'bbox': o.get('bbox')} for o in objs],
        'captions': vision_result.get('captions', [])
    }

# Planner helper: build prompt
def build_planner_prompt(instruction: str, scene_summary: Dict[str, Any]) -> str:
    obj_names = ', '.join([o['name'] for o in scene_summary.get('objects', [])]) or "none"
    prompt = textwrap.dedent(f"""
    You are a robotics task planner. The user instruction is: "{instruction}".
    The visible scene objects are: {obj_names}.
    Provide a concise step-by-step robot action plan (3-8 steps). Reference objects exactly by name.
    Output in plain text, steps numbered.
    """).strip()
    return prompt

# Planner LLM call (tries Gemini via ADK; otherwise heuristic)
def call_planner_llm(prompt: str) -> str:
    # Try real Gemini
    try:
        if GEMINI_MODEL:
            resp = GEMINI_MODEL.generate(messages=[{"role":"user","content":prompt}])
            # Extract text in different possible fields
            text = getattr(resp, "text", None) or getattr(resp, "content", None) or str(resp)
            return text if isinstance(text, str) else json.dumps(text)
    except Exception as e:
        logger.warning("Planner LLM call failed: %s", str(e))
    # Fallback heuristic plan
    # A simple rule-based generator based on objects in prompt
    objs = []
    if "The visible scene objects are:" in prompt:
        try:
            objs_part = prompt.split("The visible scene objects are:")[1].split(".")[0]
            objs = [o.strip() for o in objs_part.split(',') if o.strip() and o.strip() != "none"]
        except Exception:
            objs = []
    steps = []
    if objs:
        target = objs[0]
        steps = [
            "1. Localize the target object in the workspace using vision.",
            f"2. Navigate the manipulator to the location of {target}.",
            f"3. Execute a secure grasp of {target}.",
            "4. Move to the target placement location.",
            "5. Release the object and verify placement visually.",
            "6. Return to a safe standby position."
        ]
    else:
        steps = [
            "1. No recognizable objects detected. Perform a 360-degree scan.",
            "2. Report nothing found or wait for user clarification."
        ]
    return "\n".join(steps)

# ADK-style agents (simple classes compatible with InMemoryRunner)
class VisionAgent(Agent):
    def __init__(self, tool: VisionToolWrapper):
        super().__init__(name="VisionAgent")
        self.tool = tool

    def run(self, message: Dict[str, Any], ctx: Dict[str, Any]) -> Dict[str, Any]:
        # Expect attachments: {'image': path_or_url}
        attachments = ctx.get('attachments', {})
        image = attachments.get('image') or INPUT_IMAGE_PATH
        logger.info("VisionAgent: running detection on image=%s", str(image))
        vision_out = self.tool.detect(image)
        scene = compact_scene(vision_out)
        return {'vision_raw': vision_out, 'scene_summary': scene}

class PlannerAgent(Agent):
    def __init__(self):
        super().__init__(name="PlannerAgent")

    def run(self, message: Dict[str, Any], ctx: Dict[str, Any]) -> Dict[str, Any]:
        instruction = message.get('instruction', '')
        scene = message.get('scene_summary', {})
        prompt = build_planner_prompt(instruction, scene)
        logger.info("PlannerAgent: calling LLM for planning.")
        plan_text = call_planner_llm(prompt)
        # Sanity: check referenced objects
        obj_names = [o['name'] for o in scene.get('objects', [])]
        referenced = [n for n in obj_names if n.lower() in plan_text.lower()]
        missing = [n for n in obj_names if n not in referenced]
        return {'plan': plan_text, 'referenced_objects': referenced, 'missing_objects': missing}

class OrchestratorAgent(Agent):
    def __init__(self, vision_agent: VisionAgent, planner_agent: PlannerAgent, memory: MemoryBank):
        super().__init__(name="Orchestrator")
        self.vision_agent = vision_agent
        self.planner_agent = planner_agent
        self.memory = memory

    def run(self, message: Dict[str, Any], ctx: Dict[str, Any]) -> Dict[str, Any]:
        instruction = message.get('instruction', '')
        logger.info("Orchestrator: starting pipeline for instruction: %s", instruction)
        # Vision
        vision_res = self.vision_agent.run(message, ctx)
        # Planning
        planner_input = {'instruction': instruction, 'scene_summary': vision_res['scene_summary']}
        planner_res = self.planner_agent.run(planner_input, ctx)
        record = {
            'id': str(uuid.uuid4()),
            'timestamp': datetime.utcnow().isoformat(),
            'instruction': instruction,
            'vision': vision_res,
            'plan': planner_res
        }
        self.memory.add(record)
        logger.info("Orchestrator: pipeline finished and stored in memory.")
        return record
